In [1]:
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)

# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# CP
from utils_uqmd.utils_uq_cp import CP

# Ensure reproducibility
import random, numpy as np, torch
seed = 12345
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# ----------------------------------------------------------------------
# Data Noise
data_noise = 0.05          # same as your 2-D example


# Define the 3-D Helmholtz PDE
from utils_pde.utils_pde_3dhelmholtz import Helmholtz3D
domain = ((0.0, 1.0), (0.0, 1.0), (0.0, 1.0))   # (x0,x1),(y0,y1),(z0,z1)
k = math.pi                                     # wave number

true_solution = (
    lambda xyz: torch.sin(math.pi * xyz[..., 0:1])
              * torch.sin(math.pi * xyz[..., 1:2])
              * torch.sin(math.pi * xyz[..., 2:3])
)

pde = Helmholtz3D(k=k, domain=domain, true_solution=true_solution)

# Generate training / testing / calibration data
(X_train, Y_train)         = pde.data_generation(1_000, data_noise)
(X_test,  Y_test)          = pde.data_generation(   200, data_noise)
(X_calibration, Y_calibration) = pde.data_generation(400, data_noise)

# Number of interior collocation points for the PINN residual
colloc_pt_num = 200

# ----------------------------------------------------------------------


# Generating alphas to test
from utils_tools.utils_result_metrics import generating_alphas
alphas = generating_alphas(20)

# ----------------------------------------------------------------------

# Build base model
dist_pinn = DistanceUQPINN(
    pde_class=pde, 
    input_dim=3,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1
)

# Define tuning arguments
# Base Model
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}

# Change the parameter grids to test
fit_kwargs_grid = {
    "epochs":[200],
    "λ_pde":[3.0, 5.0], "λ_bc":[5.0, 10.0], "λ_data":[1.0, 2.0, 5.0],  # Change based on baseline model
    "lr":[1e-3, 3e-4],
    # "scheduler_cls":[StepLR], "scheduler_kwargs":[{'step_size': 5000, 'gamma': 0.5}],
    "stop_schedule":[10000, 20000, 40000]
}

baseline_pred_kwargs = {  # Change all the kwargs here
    "n_samples":20 
}

# CP Model
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",
        "k":10
}

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
    "k":1
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn,   # Change this
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "n_samples":100
}

# Defining testing grid
# --------------------------------------------
# Defining Plotting Grid
# --------------------------------------------

n_grid = 100
x = torch.linspace(domain[0][0], domain[0][1], n_grid)
y = torch.linspace(domain[1][0], domain[1][1], n_grid)
z = torch.linspace(domain[2][0], domain[2][1], n_grid)
X, Y, Z = torch.meshgrid(x, y, z, indexing='xy')
grid_test = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1), Z.reshape(-1, 1)], dim=1)

# ----------------------------------------------------------------------

# Start hyperparameter tuning 
from utils_tools.utils_tuning import hyperparameter_tuning
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage, plot_metrics_table
from utils_uqmd.utils_uq_cp import CP
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties

hyperparameter_tuning(
    plot_title="Distance CP Model", # Change this
    # Model Fitting & Predicting
    uqmodel=dist_pinn,  # Change this
    alpha=0.05, 
    X_test=grid_test, Y_test=Y_test, 
    fit_args=fit_args, fit_kwargs_grid=fit_kwargs_grid, baseline_pred_kwargs=baseline_pred_kwargs, cp_pred_kwargs=cp_pred_kwargs, 
    true_solution=pde.true_solution,
    # Coverage Test
    baseline_testing_args=baseline_testing_args, cp_testing_args=cp_testing_args,
    baseline_test_uncertainties=dist_test_uncertainties, # Change this
    # Plotting function
    plotting_func=plot_metrics_table,
    save_dir="3dhelmholtz_dismd",  # Change this
    X_validation=X_test, Y_validation=Y_test
)

Using device: cpu
Using device: cpu

[🔎] Trying: {'epochs': 200, 'λ_pde': 3.0, 'λ_bc': 5.0, 'λ_data': 1.0, 'lr': 0.001, 'stop_schedule': 10000}

[🟠] Training...


/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ep     1 | L=1.03e+02 | lr=1.0e-03

[🟠] Base Model Inferencing...


KeyboardInterrupt: 